In [ ]:
# To do
"""
- Updated the training pixels, so have to check if total sizes are similar, aggregate and create points
- See if more complicated architecture can improve results
- Test if other image with same model gives us similar results
"""

In [59]:
import ee
import folium
import geehydro
import time
import geopandas as gpd
from pyrsgis import raster
from osgeo import gdal, osr
from pyrsgis.convert import changeDimension
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [23]:
# initialize the connection to the server
ee.Initialize()

In [24]:
# Query landast archive
aoi = ee.Geometry.Rectangle([14.35, 26.96, 14.50, 27.11])
col84 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1984-04-25', '1984-06-30').filterBounds(aoi)
col93 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1993-03-20', '1993-04-30').filterBounds(aoi)
col02 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2002-03-11', '2002-03-30').filterBounds(aoi)
col11 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('2011-03-20', '2011-3-30').filterBounds(aoi)

im84 = col84.sort('SENSING_TIME').first().clip(aoi).select(ee.List.sequence(0,6))
im93 = col93.sort('SENSING_TIME').first().clip(aoi).select(ee.List.sequence(0,6))
im02 = col02.sort('SENSING_TIME').first().clip(aoi).select(ee.List.sequence(0,6))
im11 = col11.sort('SENSING_TIME').first().clip(aoi).select(ee.List.sequence(0,6))

# to get more info: collection.getInfo()

In [25]:
# Visualize landsat imagery
vizParams = {
  'bands': ['B3', 'B2', 'B1'],
  'min': 0,
  'max': 5000,
  'gamma': [0.95, 1.1, 1]}


# map of Sebha, Libya
sebhaMap = folium.Map(location=[27.039464, 14.426416], zoom_start=14)
sebhaMap.addLayer(im11,vizParams, '2011')
sebhaMap.addLayer(im02,vizParams, '2002')
sebhaMap.addLayer(im93,vizParams, '1993')
sebhaMap.addLayer(im84,vizParams, '1984')

sebhaMap.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
sebhaMap

In [26]:
"""# Export imagery function needs some love

task_config = {
    'scale': 30,  
    'region': aoi,
    'fileFormat': 'GeoTIFF',
    'crs': 4326
    }

def exportImagery(image, name, config):
    task = ee.batch.Export.image(image, name, config)
    task.start()

exportImagery(im84, 'landsat5_1984', task_config)"""

"# Export imagery function needs some love\n\ntask_config = {\n    'scale': 30,  \n    'region': aoi,\n    'fileFormat': 'GeoTIFF',\n    'crs': 4326\n    }\n\ndef exportImagery(image, name, config):\n    task = ee.batch.Export.image(image, name, config)\n    task.start()\n\nexportImagery(im84, 'landsat5_1984', task_config)"

In [28]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': im11,
    'description': 'Landsat 5 2011',
    'folder':'UrbanGrowth',
    'scale': 30,
    'fileFormat': 'GeoTIFF'
})
task.start()

while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  print('Task metadata: {})'.format(task.status()))
  time.sleep(5)

Polling for task (id: IKRBQYWLOHW57QNYL56FSAT3).
Task metadata: {'state': 'READY', 'description': 'Landsat 5 2011', 'creation_timestamp_ms': 1594804913490, 'update_timestamp_ms': 1594804913490, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'IKRBQYWLOHW57QNYL56FSAT3', 'name': 'projects/earthengine-legacy/operations/IKRBQYWLOHW57QNYL56FSAT3'})
Polling for task (id: IKRBQYWLOHW57QNYL56FSAT3).
Task metadata: {'state': 'READY', 'description': 'Landsat 5 2011', 'creation_timestamp_ms': 1594804913490, 'update_timestamp_ms': 1594804913490, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'IKRBQYWLOHW57QNYL56FSAT3', 'name': 'projects/earthengine-legacy/operations/IKRBQYWLOHW57QNYL56FSAT3'})
Polling for task (id: IKRBQYWLOHW57QNYL56FSAT3).
Task metadata: {'state': 'READY', 'description': 'Landsat 5 2011', 'creation_timestamp_ms': 1594804913490, 'update_timestamp_ms': 1594804913490, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'IKRBQYWLOHW57QNYL56FSAT3', 'n

In [123]:
# Read satellite images from google drive??


{'init': 'epsg:3857'}


In [29]:
# Read Landsat 5 image
# Read input image
image = r'C:\Users\jneuj\Dropbox\6. GIS\GeoData\SatelliteImagery\Sebha_Landsat5\Landsat 5 2002.tif'
ds1, arr = raster.read(image, bands='all')


# Get metadata of source geotiff for georeferencing for Tiff export
orig = gdal.Open(image)
gT = orig.GetGeoTransform()
proj = orig.GetProjection()

# epsg sat image was 7030 and training data is in 3857

# Check shape of sat image
initShape = arr.shape
print("Initial multispectral image shape: ", initShape)

# Reshape the image to have heigh*width rows and 2 cols (2 bands in the image and 2 input nodes)
arr = changeDimension(arr)
nBands = arr.shape[1]
print("Output multispectral image shape: ", arr.shape)

# Normalize original image
norm2 = np.amax(arr, axis=0)
arr = (arr/(norm2))

Initial multispectral image shape:  (7, 558, 500)
Output multispectral image shape:  (279000, 7)


In [30]:
print(proj)

PROJCS["WGS 84 / UTM zone 33N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32633"]]


In [31]:
# Read and prep training and validation dataset

# Read ground truth
fp = r"C:\Users\jneuj\Dropbox\6. GIS\GeoData\UrbanGrowth\train2010_pointsgood2.shp"
map_df = gpd.read_file(fp)
print(map_df.crs)

# Setup training data and validation data
xtrain = map_df[['b1_Landsat', 'b2_Landsat', 'b3_Landsat', 'b4_Landsat', 'b5_Landsat', 'b6_Landsat', 'b7_Landsat']]
ytrain = map_df[['classvalue']]

# Normalize training data
norm1 = xtrain.max()
xtrain = (xtrain/(norm1.tolist()))

{'init': 'epsg:3857'}


In [9]:
print(map_df.crs)

{'init': 'epsg:3857'}


In [33]:
# Plot training samples on sat image
# Visualize landsat imagery
vizParams = {
  'bands': ['B3', 'B2', 'B1'],
  'min': 0,
  'max': 5000,
  'gamma': [0.95, 1.1, 1]}


# map of Sebha, Libya
sebhaMap = folium.Map(location=[27.039464, 14.426416], zoom_start=14, tiles='Stamen toner')
#sebhaMap.addLayer(im11,vizParams, '2011')
sebhaMap.addLayer(im02,vizParams, '2002')
#sebhaMap.addLayer(im93,vizParams, '1993')
#sebhaMap.addLayer(im84,vizParams, '1984')


# Convert geodf to json
gjson = map_df.to_crs(epsg='4326').to_json()
points = folium.features.GeoJson(gjson)
sebhaMap.add_children(points)
    
sebhaMap.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
sebhaMap

In [43]:
# Turn the labels into an array
y = np.ravel(ytrain)
x_train, x_test, y_train, y_test = train_test_split(xtrain, y, test_size=0.30, random_state=42)

# Change data format from pandas dataframe to numpy array
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

# Reshape the data
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
arr = arr.reshape((arr.shape[0], 1, arr.shape[1]))

In [47]:
# Build an artifical neural network
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units = 12, activation = 'relu', input_shape = (1,nBands)))
#model.add(tf.keras.layers.Dense(units = 12, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(units = 10, activation = 'relu'))
model.add(tf.keras.layers.Dense(units=3, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 12)             96        
_________________________________________________________________
dropout (Dropout)            (None, 1, 12)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 10)             130       
_________________________________________________________________
dense_2 (Dense)              (None, 1, 3)              33        
Total params: 259
Trainable params: 259
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Fit the model and asses accuracy
model.fit(x_train,y_train,epochs = 70)
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

Train on 719 samples
Epoch 1/70
719/719 [==============================] - 1s 1ms/sample - loss: 1.1238 - sparse_categorical_accuracy: 0.3366
Epoch 2/70
719/719 [==============================] - 0s 50us/sample - loss: 1.0950 - sparse_categorical_accuracy: 0.3964
Epoch 3/70
719/719 [==============================] - 0s 50us/sample - loss: 1.0787 - sparse_categorical_accuracy: 0.4200
Epoch 4/70
719/719 [==============================] - 0s 57us/sample - loss: 1.0637 - sparse_categorical_accuracy: 0.4548
Epoch 5/70
719/719 [==============================] - 0s 51us/sample - loss: 1.0353 - sparse_categorical_accuracy: 0.4868
Epoch 6/70
719/719 [==============================] - 0s 51us/sample - loss: 1.0088 - sparse_categorical_accuracy: 0.5313
Epoch 7/70
719/719 [==============================] - 0s 53us/sample - loss: 1.0081 - sparse_categorical_accuracy: 0.5132
Epoch 8/70
719/719 [==============================] - 0s 54us/sample - loss: 0.9770 - sparse_categorical_accuracy: 0.5661
Epoc

In [50]:
# Model is maybe heavily overfit, but let's keep it for now
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("fashion_model.h5")

In [198]:
# Create new array with same shape as y_test, but with classified results of predicted output on x_test
yTestPredicted = model.predict(x_test)
y_test_pred = np.empty(y_test.shape)

count = 0
for i in yTestPredicted:
    print(i)
    for j in i:        
        testmax = np.amax(j, axis=-1)
        test = np.where(j == testmax)
        y_test_pred[count] = test[0]
        count += 1
y_test_pred = y_test_pred.astype(np.int)

[[0.06199856 0.929885   0.00811641]]
[[0.8121313  0.03301644 0.15485229]]
[[0.07758209 0.9136069  0.00881095]]
[[3.1454112e-02 8.0867510e-05 9.6846503e-01]]
[[0.8415435  0.04436481 0.11409172]]
[[0.15757059 0.7456361  0.0967933 ]]
[[0.32347617 0.00947609 0.66704774]]
[[0.43327063 0.03001092 0.5367184 ]]
[[0.14631909 0.00185995 0.851821  ]]
[[0.78919405 0.14021719 0.07058874]]
[[0.23074517 0.72696805 0.04228672]]
[[0.018723   0.9776277  0.00364925]]
[[0.04682466 0.92642576 0.02674962]]
[[0.07444177 0.91398525 0.01157288]]
[[0.30322576 0.00488428 0.69188994]]
[[1.9626236e-03 1.2959196e-06 9.9803609e-01]]
[[0.8169954  0.04952221 0.13348237]]
[[0.03672063 0.95731044 0.00596897]]
[[0.06818278 0.9232105  0.00860671]]
[[0.22701842 0.72550136 0.04748024]]
[[0.65161175 0.02152553 0.3268627 ]]
[[0.7660598  0.16426206 0.06967805]]
[[0.30940259 0.11637253 0.5742249 ]]
[[0.25573203 0.00254572 0.74172217]]
[[0.21364032 0.00120536 0.78515434]]
[[9.715060e-02 7.002188e-04 9.021492e-01]]
[[0.22873087 0

In [201]:
cMatrix = confusion_matrix(y_test, y_test_pred)
#pScore = precision_score(y_test, yTestPredicted)
#rScore = recall_score(y_test, yTestPredicted)

print("Confusion matrix: for 14 nodes\n", cMatrix)
#print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

Confusion matrix: for 14 nodes
 [[ 90   3  11]
 [  4 101   2]
 [  5   3  90]]


In [202]:
# Perform classifier on whole image arr
pred = model.predict(arr)
pred.shape

(279000, 1, 3)

In [203]:
pred0 = pred[:,:,0]  #Why is this different in the article?
pred1 = pred[:,:,1]
pred2 = pred[:,:,2]

In [209]:
output0 = np.reshape(pred0,  (ds1.RasterYSize, ds1.RasterXSize))
output1 = np.reshape(pred1,  (ds1.RasterYSize, ds1.RasterXSize))
output2 = np.reshape(pred2,  (ds1.RasterYSize, ds1.RasterXSize))
output0.shape

(558, 500)

In [210]:
outFile0 = r"C:\Users\jneuj\Dropbox\6. GIS\GeoData\UrbanGrowth\output\sebha\2002\builtup.tiff"
raster.export(output0, ds1, filename=outFile0, dtype='float')

outFile1 = r"C:\Users\jneuj\Dropbox\6. GIS\GeoData\UrbanGrowth\output\sebha\2002\vegetation.tiff"
raster.export(output1, ds1, filename=outFile1, dtype='float')

outFile2 = r"C:\Users\jneuj\Dropbox\6. GIS\GeoData\UrbanGrowth\output\sebha\2002\baresoil.tiff"
raster.export(output2, ds1, filename=outFile2, dtype='float')